#***Obtención de datos de tráfico - Madrid***

*Fuente:* Portal de datos abiertos del ayuntamiento de Madrid (https://datos.madrid.es/portal/site/egob).

*Conjunto de datos:* Tráfico. Histórico de datos del tráfico desde 2013 -> Tráfico. Datos del tráfico en tiempo real (años de interés 2017 a 2023).

Los datos se descargan como ficheros .csv mensuales.

***Variables originales:***

*id:* Identificador del punto de medida.

*fecha:* Fecha y hora.

*tipo_elem:*  Subárea de explotación de tráfico ('C30', 'URB', 'M30').

*intensidad:* Intensidad de número de vehículos por hora.

*ocupacion:* Porcentaje de tiempo que está un detector de tráfico ocupado por un vehículo.

*carga:* Parámetro de carga del vial. Representa una estimación del grado de congestión, calculado a partir de un algoritmo que usa como variables la intensidad y ocupación, con ciertos factores de corrección. Establece el grado de uso de la vía en un rango de 0 (vacía) a 100 (colapso).

*vmed:* Velocidad media de los vehículos detectados en el periodo de
integración.

*error:* Código de control de la validez de los datos del punto de medida.

*periodo_integracion:* X tiempo de toma de datos.


In [93]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

***Exploración de conjunto mensual y extrapolación al resto***

In [ ]:
path = '/content/drive/MyDrive/madrid_traffic_data/original/2021/01-2021.csv'

In [ ]:
data = pd.read_csv(path, sep=';', header=0)
data.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,2021-01-01 00:00:00,M30,144,NaN,0,55.0,N,5
1,1001,2021-01-01 00:15:00,M30,84,0.0,0,56.0,N,5
2,1001,2021-01-01 00:30:00,M30,108,0.0,0,55.0,N,5
3,1001,2021-01-01 00:45:00,M30,792,1.0,0,59.0,N,5
4,1001,2021-01-01 01:00:00,M30,1704,4.0,0,61.0,N,5


In [ ]:
data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10422264 entries, 0 to 10422263
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 715.6+ MB


In [ ]:
data.isnull().sum()

id                          0
fecha                       0
tipo_elem                   0
intensidad                  0
ocupacion               34803
carga                       0
vmed                     6419
error                  240860
periodo_integracion         0
dtype: int64

In [ ]:
data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10422264 entries, 0 to 10422263
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          int64  
 1   fecha       object 
 2   tipo_elem   object 
 3   intensidad  int64  
 4   ocupacion   float64
 5   carga       int64  
 6   vmed        float64
dtypes: float64(2), int64(3), object(2)
memory usage: 556.6+ MB


In [ ]:
data.duplicated().sum() # sin duplicados

0

In [ ]:
data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
data.isnull().sum()

id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64

In [ ]:
data.shape

(10381042, 7)

In [ ]:
data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora
data.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed
1,1001,2021-01-01,NO_URB,84,0.0,0,56.0
2,1001,2021-01-01,NO_URB,108,0.0,0,55.0
3,1001,2021-01-01,NO_URB,792,1.0,0,59.0
4,1001,2021-01-01,NO_URB,1704,4.0,0,61.0
5,1001,2021-01-01,NO_URB,2388,6.0,0,58.0


***Selección de puntos de medida de tráfico por estación CA de interes***

In [ ]:
ptos_t = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/ptos_x_est_CA_2km.csv')
ptos_t.head()

,Unnamed: 0,ptos_RC,ptos_B,ptos_CC,ptos_R,ptos_PE,ptos_B_ext,ptos_CC_ext
0,0,3909,6334.0,3844.0,3842.0,4081.0,6332.0,3840
1,1,3910,6335.0,3925.0,3843.0,3614.0,6333.0,3841
2,2,3911,6336.0,3930.0,3845.0,4167.0,6334.0,3842
3,3,3460,6337.0,3931.0,3850.0,4168.0,6335.0,3843
4,4,3461,6338.0,3937.0,7029.0,4169.0,6336.0,3844


**Barajas**

In [ ]:
data_B = data[data.id.isin(list(ptos_t.ptos_B))]
data_B.drop(['id'], axis=1, inplace=True)
print(data_B.tipo_elem.value_counts())

URB    47119
Name: tipo_elem, dtype: int64


In [ ]:
data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
data_B_m.drop('tipo_elem', axis=1, inplace=True)
data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
data_B_m.insert(0,'estacion','BARAJAS')
data_B_m

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
0,BARAJAS,2021-01-01,103.356207,2.974423,8.748596,0.0
1,BARAJAS,2021-01-02,142.653917,4.133251,12.696484,0.0
2,BARAJAS,2021-01-03,129.061958,3.628253,11.324040,0.0
3,BARAJAS,2021-01-04,190.639093,6.006127,17.204657,0.0
4,BARAJAS,2021-01-05,193.408952,6.239730,17.587983,0.0
5,BARAJAS,2021-01-06,127.104450,4.365266,11.599506,0.0
6,BARAJAS,2021-01-07,163.611281,5.692213,15.073574,0.0
7,BARAJAS,2021-01-08,135.590174,4.396144,11.963308,0.0
8,BARAJAS,2021-01-09,19.915416,0.343793,1.437926,0.0
9,BARAJAS,2021-01-10,43.783920,1.437186,2.912060,0.0


***Cuatro Caminos***

In [ ]:
data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
data_CC.drop(['id'], axis=1, inplace=True)
print(data_CC.tipo_elem.value_counts())

URB    998730
Name: tipo_elem, dtype: int64


In [ ]:
data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
data_CC_m.drop('tipo_elem', axis=1, inplace=True)
data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
data_CC_m

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
0,CUATRO CAMINOS,2021-01-01,187.536089,1.978696,8.076892,0.0
1,CUATRO CAMINOS,2021-01-02,267.373629,2.931034,11.653850,0.0
2,CUATRO CAMINOS,2021-01-03,239.742297,2.545870,10.424016,0.0
3,CUATRO CAMINOS,2021-01-04,403.426727,4.735660,17.861832,0.0
4,CUATRO CAMINOS,2021-01-05,385.516512,4.571198,17.312375,0.0
5,CUATRO CAMINOS,2021-01-06,238.345327,2.438386,10.412336,0.0
6,CUATRO CAMINOS,2021-01-07,337.817480,3.938288,15.172530,0.0
7,CUATRO CAMINOS,2021-01-08,270.064452,3.602954,12.333306,0.0
8,CUATRO CAMINOS,2021-01-09,29.759556,1.387999,2.047021,0.0
9,CUATRO CAMINOS,2021-01-10,52.783770,1.523295,2.732389,0.0


***Retiro***

In [ ]:
data_R = data[data.id.isin(list(ptos_t.ptos_R))]
data_R.drop(['id'], axis=1, inplace=True)
print(data_R.tipo_elem.value_counts())

URB       850614
NO_URB    118057
Name: tipo_elem, dtype: int64


In [ ]:
data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
data_R_m.drop('tipo_elem', axis=1, inplace=True)
data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
data_R_m['vmed_noURB'] = vmed_noURB
data_R_m.insert(0,'estacion','RETIRO')
data_R_m

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
0,RETIRO,2021-01-01,248.050256,2.653806,10.757992,67.271271
1,RETIRO,2021-01-02,350.310435,4.072448,15.596162,65.803860
2,RETIRO,2021-01-03,322.138978,3.711456,14.363939,66.124639
3,RETIRO,2021-01-04,538.369217,6.418433,23.482414,65.532393
4,RETIRO,2021-01-05,482.528546,5.821633,21.309829,64.666234
5,RETIRO,2021-01-06,301.914587,3.372638,13.241203,65.362776
6,RETIRO,2021-01-07,431.750937,5.099892,18.901016,63.674961
7,RETIRO,2021-01-08,366.594284,4.577618,15.521182,53.221159
8,RETIRO,2021-01-09,40.463845,0.716414,1.651551,3.926892
9,RETIRO,2021-01-10,49.132969,1.118067,2.571114,11.403670


***Plaza Elíptica***

In [ ]:
data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
data_PE.drop(['id'], axis=1, inplace=True)
print(data_PE.tipo_elem.value_counts())

URB       704904
NO_URB    138339
Name: tipo_elem, dtype: int64


In [ ]:
data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
data_PE_m.drop('tipo_elem', axis=1, inplace=True)
data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
data_PE_m['vmed_noURB'] = vmed_noURB
data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
data_PE_m

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
0,PLAZA ELIPTICA,2021-01-01,221.936903,3.613783,9.238317,50.513465
1,PLAZA ELIPTICA,2021-01-02,286.463533,4.968983,12.048588,49.799326
2,PLAZA ELIPTICA,2021-01-03,264.060575,4.293846,10.777824,49.530480
3,PLAZA ELIPTICA,2021-01-04,393.913611,6.975605,15.863019,50.595924
4,PLAZA ELIPTICA,2021-01-05,400.385818,7.204819,16.257338,49.833844
5,PLAZA ELIPTICA,2021-01-06,275.609315,4.847619,11.459044,49.649111
6,PLAZA ELIPTICA,2021-01-07,358.724487,6.516917,14.466012,50.195509
7,PLAZA ELIPTICA,2021-01-08,328.864054,6.002553,12.386102,51.834981
8,PLAZA ELIPTICA,2021-01-09,68.916146,2.043287,4.070625,4.308366
9,PLAZA ELIPTICA,2021-01-10,59.546786,1.519716,3.390754,10.968511


Dos estaciones con *vmed_noURB* (Retiro y Plaza Eliptica, con elementos 'M30' y 'C30') y dos sin (Barajas y Cuatro Caminos).

In [ ]:
data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
data_month

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
0,BARAJAS,2021-01-01,103.356207,2.974423,8.748596,0.000000
1,BARAJAS,2021-01-02,142.653917,4.133251,12.696484,0.000000
2,BARAJAS,2021-01-03,129.061958,3.628253,11.324040,0.000000
3,BARAJAS,2021-01-04,190.639093,6.006127,17.204657,0.000000
4,BARAJAS,2021-01-05,193.408952,6.239730,17.587983,0.000000
...,...,...,...,...,...,...
26,PLAZA ELIPTICA,2021-01-27,410.263865,7.134681,16.011383,47.204421
27,PLAZA ELIPTICA,2021-01-28,422.789986,7.496791,16.529513,47.212295
28,PLAZA ELIPTICA,2021-01-29,428.665626,7.595468,17.079735,48.003511
29,PLAZA ELIPTICA,2021-01-30,301.736781,5.340193,12.894389,48.172538


In [ ]:
data_month.isnull().sum()

estacion      0
fecha         0
intensidad    0
ocupacion     0
carga         0
vmed_noURB    0
dtype: int64

In [ ]:
data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/01-2021_mod_points.csv', index=False) # guardar

***Procesamiento mes a mes en ciclo***

In [ ]:
# 2021
path = '/content/drive/MyDrive/madrid_traffic_data/original/2021/'
d_month = ['02-2021', '03-2021', '04-2021', '05-2021', '06-2021', '07-2021',
           '08-2021', '09-2021', '10-2021', '11-2021', '12-2021']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  print(data_month.isnull().sum())
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10477478 entries, 0 to 10477477
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 719.4+ MB
None
id                          0
fecha                       0
tipo_elem                   0
intensidad                  0
ocupacion               39627
carga                       0
vmed                     5151
error                  150953
periodo_integracion         0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(10432700, 7)
BARAJAS
URB    45497


In [ ]:
# 2022
path = '/content/drive/MyDrive/madrid_traffic_data/original/2022/'
d_month = ['01-2022', '02-2022', '03-2022', '04-2022', '05-2022', '06-2022',
           '07-2022', '08-2022', '09-2022', '10-2022', '11-2022', '12-2022']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12429813 entries, 0 to 12429812
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 853.5+ MB
None
id                          0
fecha                       0
tipo_elem                   0
intensidad                  0
ocupacion               26253
carga                       0
vmed                    10439
error                  109582
periodo_integracion         0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(12393121, 7)
BARAJAS
URB    48348


In [ ]:
# 2023
path = '/content/drive/MyDrive/madrid_traffic_data/original/2023/'
d_month = ['01-2023', '02-2023', '03-2023', '04-2023', '05-2023', '06-2023',
           '07-2023', '08-2023', '09-2023', '10-2023', '11-2023', '12-2023']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12644350 entries, 0 to 12644349
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 868.2+ MB
None
id                         0
fecha                      0
tipo_elem                  0
intensidad                 0
ocupacion              31070
carga                      0
vmed                    3475
error                  25882
periodo_integracion        0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(12609805, 7)
BARAJAS
URB    53185
Name: tip

In [ ]:
# 2020
path = '/content/drive/MyDrive/madrid_traffic_data/original/2020/'
d_month = ['01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020',
           '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11577408 entries, 0 to 11577407
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 795.0+ MB
None
id                         0
fecha                      0
tipo_elem                  0
intensidad                 0
ocupacion              24484
carga                      0
vmed                    7779
error                      0
periodo_integracion        0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(11545145, 7)
BARAJAS
URB    50133
Name: tip

In [ ]:
# 2019
path = '/content/drive/MyDrive/madrid_traffic_data/original/2019/'
d_month = ['01-2019', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019',
           '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11203627 entries, 0 to 11203626
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 769.3+ MB
None
id                         0
fecha                      0
tipo_elem                  0
intensidad                 0
ocupacion              23069
carga                      0
vmed                    2546
error                      0
periodo_integracion        0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(11178012, 7)
BARAJAS
URB    50402
Name: tip

In [ ]:
# 2018-1
path = '/content/drive/MyDrive/madrid_traffic_data/original/2018/'
d_month = ['01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018',
           '07-2018', '08-2018', '09-2018', '10-2018', '11-2018', '12-2018']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/' + e + '_mod_points.csv', index=False) # guardar

In [ ]:
# 2018-2
data = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/original/2018/11-2018.csv', sep=';', header=0) # hecho también para 09 y 10/2018 individualmente
data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
print(data.info())
print(data.isnull().sum())
data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
#data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
print(data.isnull().sum())
print(data.shape)
data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
print('BARAJAS')
data_B = data[data.id.isin(list(ptos_t.ptos_B))]
data_B.drop(['id'], axis=1, inplace=True)
print(data_B.tipo_elem.value_counts())
data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=False) # sin elementos no_URB -> sin vmed
data_B_m.drop('tipo_elem', axis=1, inplace=True)
data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
data_B_m.insert(0,'estacion','BARAJAS')
print(data_B_m.head())
print(data_B_m.isnull().sum())

# Cuatro Caminos
print('CUATRO CAMINOS')
data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
data_CC.drop(['id'], axis=1, inplace=True)
print(data_CC.tipo_elem.value_counts())
data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=False) # sin elementos no_URB -> sin vmed
data_CC_m.drop('tipo_elem', axis=1, inplace=True)
data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
print(data_CC_m.head())
print(data_CC_m.isnull().sum())

# Retiro
print('RETIRO')
data_R = data[data.id.isin(list(ptos_t.ptos_R))]
data_R.drop(['id'], axis=1, inplace=True)
print(data_R.tipo_elem.value_counts())
data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
data_R_m.drop('tipo_elem', axis=1, inplace=True)
data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
data_R_m['vmed_noURB'] = vmed_noURB
data_R_m.insert(0,'estacion','RETIRO')
print(data_R_m.head())
print(data_R_m.isnull().sum())
del(vmed_noURB)

# Plaza Elíptica
print('PLAZA ELIPTICA')
data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
data_PE.drop(['id'], axis=1, inplace=True)
print(data_PE.tipo_elem.value_counts())
data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
data_PE_m.drop('tipo_elem', axis=1, inplace=True)
data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
data_PE_m['vmed_noURB'] = vmed_noURB
data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
print(data_PE_m.head())
print(data_PE_m.isnull().sum())

# Mes
data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/11-2018_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10703589 entries, 0 to 10703588
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           float64
 4   ocupacion            float64
 5   carga                float64
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 735.0+ MB
None
id                           0
fecha                        0
tipo_elem                    0
intensidad              221371
ocupacion              2264058
carga                   790753
vmed                   9552770
error                        0
periodo_integracion          0
dtype: int64
id                  0
fecha               0
tipo_elem           0
intensidad     221371
ocupacion     2264058
carga          790753
vmed          95527

In [ ]:
data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/11-2018_mod_points.csv', index=False) # guardar

In [ ]:
# 2017-1
path = '/content/drive/MyDrive/madrid_traffic_data/original/2017/'
d_month = ['01-2017', '02-2017']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['PUNTOS MEDIDA URBANOS', 'PUNTOS MEDIDA M-30'],['URB', 'NO_URB'],inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['identif', 'error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.idelem.isin(list(ptos_t.ptos_B))]
  data_B.drop(['idelem'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.idelem.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['idelem'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.idelem.isin(list(ptos_t.ptos_R))]
  data_R.drop(['idelem'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.idelem.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['idelem'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9843784 entries, 0 to 9843783
Data columns (total 10 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   idelem               int64 
 1   fecha                object
 2   identif              object
 3   tipo_elem            object
 4   intensidad           int64 
 5   ocupacion            int64 
 6   carga                int64 
 7   vmed                 int64 
 8   error                object
 9   periodo_integracion  int64 
dtypes: int64(6), object(4)
memory usage: 751.0+ MB
None
idelem                 0
fecha                  0
identif                0
tipo_elem              0
intensidad             0
ocupacion              0
carga                  0
vmed                   0
error                  0
periodo_integracion    0
dtype: int64
idelem        0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(9843784, 7)
BARAJAS
URB    50393
Name: tipo_e

In [ ]:
# 2017-2
path = '/content/drive/MyDrive/madrid_traffic_data/original/2017/'
d_month = ['03-2017']

for e in d_month:

  data = pd.read_csv(path + e + '.csv')
  data.tipo_elem.replace(['PUNTOS MEDIDA URBANOS','PUNTOS MEDIDA M-30'], ['URB', 'NO_URB'],inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['identif', 'error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.idelem.isin(list(ptos_t.ptos_B))]
  data_B.drop(['idelem'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.idelem.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['idelem'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.idelem.isin(list(ptos_t.ptos_R))]
  data_R.drop(['idelem'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.idelem.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['idelem'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10933541 entries, 0 to 10933540
Data columns (total 10 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   idelem               int64 
 1   fecha                object
 2   identif              object
 3   tipo_elem            object
 4   intensidad           int64 
 5   ocupacion            int64 
 6   carga                int64 
 7   vmed                 int64 
 8   error                object
 9   periodo_integracion  int64 
dtypes: int64(6), object(4)
memory usage: 834.2+ MB
None
idelem                 0
fecha                  0
identif                0
tipo_elem              0
intensidad             0
ocupacion              0
carga                  0
vmed                   0
error                  0
periodo_integracion    0
dtype: int64
idelem        0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(10933541, 7)
BARAJAS
URB    49572
Name: tip

In [ ]:
# 2017-3
path = '/content/drive/MyDrive/madrid_traffic_data/original/2017/'
d_month = ['04-2017', '05-2017', '06-2017', '07-2017', '08-2017', '09-2017']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['PUNTOS MEDIDA URBANOS','PUNTOS MEDIDA M-30'], ['URB', 'NO_URB'],inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['identif', 'error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.idelem.isin(list(ptos_t.ptos_B))]
  data_B.drop(['idelem'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.idelem.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['idelem'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.idelem.isin(list(ptos_t.ptos_R))]
  data_R.drop(['idelem'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.idelem.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['idelem'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10619901 entries, 0 to 10619900
Data columns (total 10 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   idelem               int64 
 1   fecha                object
 2   identif              object
 3   tipo_elem            object
 4   intensidad           int64 
 5   ocupacion            int64 
 6   carga                int64 
 7   vmed                 int64 
 8   error                object
 9   periodo_integracion  int64 
dtypes: int64(6), object(4)
memory usage: 810.2+ MB
None
idelem                 0
fecha                  0
identif                0
tipo_elem              0
intensidad             0
ocupacion              0
carga                  0
vmed                   0
error                  0
periodo_integracion    0
dtype: int64
idelem        0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(10619901, 7)
BARAJAS
URB    45414
Name: tip

In [ ]:
# 2017-4
path = '/content/drive/MyDrive/madrid_traffic_data/original/2017/'
d_month = ['10-2017', '11-2017', '12-2017']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['PUNTOS MEDIDA URBANOS','PUNTOS MEDIDA M-30'], ['URB', 'NO_URB'],inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11058917 entries, 0 to 11058916
Data columns (total 9 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   id                   int64 
 1   fecha                object
 2   tipo_elem            object
 3   intensidad           int64 
 4   ocupacion            int64 
 5   carga                int64 
 6   vmed                 int64 
 7   error                object
 8   periodo_integracion  int64 
dtypes: int64(6), object(3)
memory usage: 759.4+ MB
None
id                     0
fecha                  0
tipo_elem              0
intensidad             0
ocupacion              0
carga                  0
vmed                   0
error                  0
periodo_integracion    0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(11058917, 7)
BARAJAS
URB    50154
Name: tipo_elem, dtype: int64
  estacion       fecha  intensidad  oc

In [ ]:
# 2017-5
path = '/content/drive/MyDrive/madrid_traffic_data/original/2017/'
d_month = ['12-2017']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['PUNTOS MEDIDA URBANOS','PUNTOS MEDIDA M-30'], ['URB', 'NO_URB'],inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())
  data_B_mm = data_B_m.append(fill_b)                      # obtenidos más adelante
  data_B_mm['fecha']= pd.to_datetime(data_B_mm['fecha'])
  dataB = data_B_mm.sort_values(['fecha']).reset_index(drop=True)


  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())
  data_CC_mm = data_CC_m.append(fill_cc)
  data_CC_mm['fecha']= pd.to_datetime(data_CC_mm['fecha'])
  dataCC = data_CC_mm.sort_values(['fecha']).reset_index(drop=True)

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([dataB, dataCC, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10299144 entries, 0 to 10299143
Data columns (total 9 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   id                   int64 
 1   fecha                object
 2   tipo_elem            object
 3   intensidad           int64 
 4   ocupacion            int64 
 5   carga                int64 
 6   vmed                 int64 
 7   error                object
 8   periodo_integracion  int64 
dtypes: int64(6), object(3)
memory usage: 707.2+ MB
None
id                     0
fecha                  0
tipo_elem              0
intensidad             0
ocupacion              0
carga                  0
vmed                   0
error                  0
periodo_integracion    0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(10299144, 7)
BARAJAS
URB    44547
Name: tipo_elem, dtype: int64
  estacion       fecha  intensidad  oc

In [ ]:
# 2017-5
path = '/content/drive/MyDrive/madrid_traffic_data/original/2017/12-2017.csv'

data = pd.read_csv(path, sep=';', header=0)
data.tipo_elem.replace(['PUNTOS MEDIDA URBANOS','PUNTOS MEDIDA M-30'], ['URB', 'NO_URB'],inplace=True) # unificar nomenclatura
print(data.info())
print(data.isnull().sum())
data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
print(data.isnull().sum())
print(data.shape)
data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

# Barajas
print('BARAJAS')
data_B = data[data.id.isin(list(ptos_t.ptos_B))]
data_B.drop(['id'], axis=1, inplace=True)
print(data_B.tipo_elem.value_counts())
data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
data_B_m.drop('tipo_elem', axis=1, inplace=True)
data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
data_B_m.insert(0,'estacion','BARAJAS')
print(data_B_m.head())
print(data_B_m.isnull().sum())
data_B_mm = data_B_m.append(fill_b)                      # obtenidos más adelante
data_B_mm['fecha']= pd.to_datetime(data_B_mm['fecha'])
dataB = data_B_mm.sort_values(['fecha']).reset_index(drop=True)
dataB.fecha = pd.to_datetime(dataB.fecha).dt.date

# Cuatro Caminos
print('CUATRO CAMINOS')
data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
data_CC.drop(['id'], axis=1, inplace=True)
print(data_CC.tipo_elem.value_counts())
data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
data_CC_m.drop('tipo_elem', axis=1, inplace=True)
data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
print(data_CC_m.head())
print(data_CC_m.isnull().sum())
data_CC_mm = data_CC_m.append(fill_cc)
data_CC_mm['fecha']= pd.to_datetime(data_CC_mm['fecha'])
dataCC = data_CC_mm.sort_values(['fecha']).reset_index(drop=True)
dataCC.fecha = pd.to_datetime(dataCC.fecha).dt.date

# Retiro
print('RETIRO')
data_R = data[data.id.isin(list(ptos_t.ptos_R))]
data_R.drop(['id'], axis=1, inplace=True)
print(data_R.tipo_elem.value_counts())
data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
data_R_m.drop('tipo_elem', axis=1, inplace=True)
data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
data_R_m['vmed_noURB'] = vmed_noURB
data_R_m.insert(0,'estacion','RETIRO')
print(data_R_m.head())
print(data_R_m.isnull().sum())
del(vmed_noURB)

# Plaza Elíptica
print('PLAZA ELIPTICA')
data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
data_PE.drop(['id'], axis=1, inplace=True)
print(data_PE.tipo_elem.value_counts())
data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
data_PE_m.drop('tipo_elem', axis=1, inplace=True)
data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
data_PE_m['vmed_noURB'] = vmed_noURB
data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
print(data_PE_m.head())
print(data_PE_m.isnull().sum())

# Mes
data_R_m.carga.replace(0, data_R_m.carga.mean(), inplace=True)
data_PE_m.carga.replace(0, data_PE_m.carga.mean(), inplace=True)
data_month1 = pd.concat([dataB, dataCC, data_R_m, data_PE_m])
data_month1.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/12-2017_mod_points.csv', index=False) # guardar

***Unir datos por año***

In [ ]:
# 2017
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/01-2017_mod_points.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/02-2017_mod_points.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/03-2017_mod_points.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/04-2017_mod_points.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/05-2017_mod_points.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/06-2017_mod_points.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/07-2017_mod_points.csv')
d8 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/08-2017_mod_points.csv')
d9 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/09-2017_mod_points.csv')
d10 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/10-2017_mod_points.csv')
d11 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/11-2017_mod_points.csv')
d12 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2017/12-2017_mod_points.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2017.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    1460 non-null   object 
 1   fecha       1460 non-null   object 
 2   intensidad  1460 non-null   float64
 3   ocupacion   1460 non-null   float64
 4   carga       1460 non-null   float64
 5   vmed_noURB  1460 non-null   float64
dtypes: float64(4), object(2)
memory usage: 68.6+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
1455,PLAZA ELIPTICA,2017-12-27,425.640958,7.425683,15.657143,44.406765
1456,PLAZA ELIPTICA,2017-12-28,436.703439,7.749512,16.383665,45.360145
1457,PLAZA ELIPTICA,2017-12-29,452.452880,8.043498,17.164155,45.647548
1458,PLAZA ELIPTICA,2017-12-30,376.485036,6.124927,14.418642,47.246173
1459,PLAZA ELIPTICA,2017-12-31,318.866398,5.438901,12.696136,46.286352


In [ ]:
# 2018
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/01-2018_mod_points.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/02-2018_mod_points.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/03-2018_mod_points.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/04-2018_mod_points.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/05-2018_mod_points.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/06-2018_mod_points.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/07-2018_mod_points.csv')
d8 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/08-2018_mod_points.csv')
d9 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/09-2018_mod_points.csv')
d10 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/10-2018_mod_points.csv')
d11 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/11-2018_mod_points.csv')
d12 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2018/12-2018_mod_points.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2018.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    1460 non-null   object 
 1   fecha       1460 non-null   object 
 2   intensidad  1460 non-null   float64
 3   ocupacion   1460 non-null   float64
 4   carga       1460 non-null   float64
 5   vmed_noURB  1216 non-null   float64
dtypes: float64(4), object(2)
memory usage: 68.6+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
1455,PLAZA ELIPTICA,2018-12-27,457.742403,7.760590,17.101651,49.608620
1456,PLAZA ELIPTICA,2018-12-28,462.001985,8.198213,17.807704,51.452585
1457,PLAZA ELIPTICA,2018-12-29,392.793156,6.490773,14.443759,53.775433
1458,PLAZA ELIPTICA,2018-12-30,358.324405,5.647862,12.780716,53.736499
1459,PLAZA ELIPTICA,2018-12-31,386.000750,6.840782,15.247888,53.084708


In [ ]:
# 2019
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/01-2019_mod_points.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/02-2019_mod_points.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/03-2019_mod_points.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/04-2019_mod_points.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/05-2019_mod_points.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/06-2019_mod_points.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/07-2019_mod_points.csv')
d8 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/08-2019_mod_points.csv')
d9 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/09-2019_mod_points.csv')
d10 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/10-2019_mod_points.csv')
d11 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/11-2019_mod_points.csv')
d12 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2019/12-2019_mod_points.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2019.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    1460 non-null   object 
 1   fecha       1460 non-null   object 
 2   intensidad  1460 non-null   float64
 3   ocupacion   1460 non-null   float64
 4   carga       1460 non-null   float64
 5   vmed_noURB  1460 non-null   float64
dtypes: float64(4), object(2)
memory usage: 68.6+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
1455,PLAZA ELIPTICA,2019-12-27,439.690528,7.744150,16.790580,52.753384
1456,PLAZA ELIPTICA,2019-12-28,367.834115,6.696138,14.902657,53.369123
1457,PLAZA ELIPTICA,2019-12-29,336.636499,5.751034,13.285480,53.114940
1458,PLAZA ELIPTICA,2019-12-30,430.307060,7.949462,17.082803,51.831361
1459,PLAZA ELIPTICA,2019-12-31,364.049588,6.584691,14.852205,50.777292


In [ ]:
# 2020
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/01-2020_mod_points.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/02-2020_mod_points.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/03-2020_mod_points.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/04-2020_mod_points.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/05-2020_mod_points.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/06-2020_mod_points.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/07-2020_mod_points.csv')
d8 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/08-2020_mod_points.csv')
d9 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/09-2020_mod_points.csv')
d10 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/10-2020_mod_points.csv')
d11 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/11-2020_mod_points.csv')
d12 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2020/12-2020_mod_points.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2020.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    1464 non-null   object 
 1   fecha       1464 non-null   object 
 2   intensidad  1464 non-null   float64
 3   ocupacion   1464 non-null   float64
 4   carga       1464 non-null   float64
 5   vmed_noURB  1464 non-null   float64
dtypes: float64(4), object(2)
memory usage: 68.8+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
1459,PLAZA ELIPTICA,2020-12-27,257.127831,4.015670,10.343115,49.897161
1460,PLAZA ELIPTICA,2020-12-28,383.265278,6.614582,15.247392,50.719306
1461,PLAZA ELIPTICA,2020-12-29,394.512585,6.884139,15.698824,50.049747
1462,PLAZA ELIPTICA,2020-12-30,407.880481,7.293642,16.383315,49.939183
1463,PLAZA ELIPTICA,2020-12-31,325.049463,5.925890,13.860532,50.079337


In [ ]:
# 2021
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/01-2021_mod_points.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/02-2021_mod_points.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/03-2021_mod_points.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/04-2021_mod_points.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/05-2021_mod_points.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/06-2021_mod_points.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/07-2021_mod_points.csv')
d8 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/08-2021_mod_points.csv')
d9 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/09-2021_mod_points.csv')
d10 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/10-2021_mod_points.csv')
d11 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/11-2021_mod_points.csv')
d12 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2021/12-2021_mod_points.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2021.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    1460 non-null   object 
 1   fecha       1460 non-null   object 
 2   intensidad  1460 non-null   float64
 3   ocupacion   1460 non-null   float64
 4   carga       1460 non-null   float64
 5   vmed_noURB  1460 non-null   float64
dtypes: float64(4), object(2)
memory usage: 68.6+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
1455,PLAZA ELIPTICA,2021-12-27,360.929413,5.976506,13.957789,50.794669
1456,PLAZA ELIPTICA,2021-12-28,375.587629,5.882644,14.220520,50.200842
1457,PLAZA ELIPTICA,2021-12-29,376.067143,6.142127,14.578697,49.442098
1458,PLAZA ELIPTICA,2021-12-30,395.256767,6.486777,15.335202,50.161234
1459,PLAZA ELIPTICA,2021-12-31,322.582790,5.111803,12.784641,50.687820


In [ ]:
# 2022
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/01-2022_mod_points.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/02-2022_mod_points.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/03-2022_mod_points.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/04-2022_mod_points.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/05-2022_mod_points.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/06-2022_mod_points.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/07-2022_mod_points.csv')
d8 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/08-2022_mod_points.csv')
d9 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/09-2022_mod_points.csv')
d10 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/10-2022_mod_points.csv')
d11 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/11-2022_mod_points.csv')
d12 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2022/12-2022_mod_points.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2022.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    1460 non-null   object 
 1   fecha       1460 non-null   object 
 2   intensidad  1460 non-null   float64
 3   ocupacion   1460 non-null   float64
 4   carga       1460 non-null   float64
 5   vmed_noURB  1460 non-null   float64
dtypes: float64(4), object(2)
memory usage: 68.6+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
1455,PLAZA ELIPTICA,2022-12-27,406.107708,6.752433,15.052394,55.378850
1456,PLAZA ELIPTICA,2022-12-28,412.979238,6.843338,15.129123,55.346014
1457,PLAZA ELIPTICA,2022-12-29,420.496568,7.006344,15.708787,55.377264
1458,PLAZA ELIPTICA,2022-12-30,428.933533,7.157480,16.153829,55.438406
1459,PLAZA ELIPTICA,2022-12-31,372.059995,5.678215,13.098411,55.500679


In [ ]:
# 2023
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/01-2023_mod_points.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/02-2023_mod_points.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/03-2023_mod_points.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/04-2023_mod_points.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/05-2023_mod_points.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/06-2023_mod_points.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/07-2023_mod_points.csv')
d8 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/08-2023_mod_points.csv')
d9 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/09-2023_mod_points.csv')
d10 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/10-2023_mod_points.csv')
d11 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/11-2023_mod_points.csv')
d12 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/12-2023_mod_points.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2023.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    1460 non-null   object 
 1   fecha       1460 non-null   object 
 2   intensidad  1460 non-null   float64
 3   ocupacion   1460 non-null   float64
 4   carga       1460 non-null   float64
 5   vmed_noURB  1460 non-null   float64
dtypes: float64(4), object(2)
memory usage: 68.6+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
1455,PLAZA ELIPTICA,2023-12-27,442.610851,7.392163,14.070328,44.468531
1456,PLAZA ELIPTICA,2023-12-28,452.437417,7.582276,14.243825,43.262400
1457,PLAZA ELIPTICA,2023-12-29,448.369909,7.503702,14.368563,44.952612
1458,PLAZA ELIPTICA,2023-12-30,368.677090,5.912876,12.088462,46.555256
1459,PLAZA ELIPTICA,2023-12-31,318.645963,5.065637,10.594595,45.380719


En este punto se revisaron los ficheros mensuales de 2017, 2018 y 2023 dado las filas faltantes. Reprocesamiento individual de meses con datos faltantes: 12/17 (celda #2017-5), 9-11/18 (celdas #2018) y 12/23. En el caso de 12/17 faltaban valores de 6-7-8/12/2017, se completaron con los valores del año más cercano (2018) -> mayor probabilidad de silmilitud en el tráfico.
Posteriormente se volvieron a generar los ficheros anuales sin datos faltantes.

In [ ]:
# 2023-12
path = '/content/drive/MyDrive/madrid_traffic_data/original/2023/'
d_month = ['12-2023']

for e in d_month:

  data = pd.read_csv((path + e + '.csv'), sep=';', header=0)
  data.tipo_elem.replace(['M30','C30'],'NO_URB',inplace=True) # unificar nomenclatura
  print(data.info())
  print(data.isnull().sum())
  data.drop(['error', 'periodo_integracion'], axis=1, inplace=True) # columnas de no interés
  data.drop_duplicates(inplace=True) # descartar registros duplicados si existieran
  data.dropna(inplace=True) # descartar registros nulos ya que son mínimos
  print(data.isnull().sum())
  print(data.shape)
  data.fecha = pd.to_datetime(data.fecha).dt.date # eliminar hora

  # Barajas
  print('BARAJAS')
  data_B = data[data.id.isin(list(ptos_t.ptos_B_ext))]
  data_B.drop(['id'], axis=1, inplace=True)
  print(data_B.tipo_elem.value_counts())
  data_B_m = data_B.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_B_m.drop('tipo_elem', axis=1, inplace=True)
  data_B_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_B_m.insert(0,'estacion','BARAJAS')
  print(data_B_m.head())
  print(data_B_m.isnull().sum())

  # Cuatro Caminos
  print('CUATRO CAMINOS')
  data_CC = data[data.id.isin(list(ptos_t.ptos_CC))]
  data_CC.drop(['id'], axis=1, inplace=True)
  print(data_CC.tipo_elem.value_counts())
  data_CC_m = data_CC.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True) # sin elementos no_URB -> sin vmed
  data_CC_m.drop('tipo_elem', axis=1, inplace=True)
  data_CC_m.rename(columns={'vmed':'vmed_noURB'}, inplace=True)
  data_CC_m.insert(0,'estacion','CUATRO CAMINOS')
  print(data_CC_m.head())
  print(data_CC_m.isnull().sum())

  # Retiro
  print('RETIRO')
  data_R = data[data.id.isin(list(ptos_t.ptos_R))]
  data_R.drop(['id'], axis=1, inplace=True)
  print(data_R.tipo_elem.value_counts())
  data_R_m = data_R.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_R_m[data_R_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_R_m.drop('tipo_elem', axis=1, inplace=True)
  data_R_m = data_R.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_R_m['vmed_noURB'] = vmed_noURB
  data_R_m.insert(0,'estacion','RETIRO')
  print(data_R_m.head())
  print(data_R_m.isnull().sum())
  del(vmed_noURB)

  # Plaza Elíptica
  print('PLAZA ELIPTICA')
  data_PE = data[data.id.isin(list(ptos_t.ptos_PE))]
  data_PE.drop(['id'], axis=1, inplace=True)
  print(data_PE.tipo_elem.value_counts())
  data_PE_m = data_PE.groupby(['fecha','tipo_elem'], as_index=False).mean(numeric_only=True)
  vmed_noURB = data_PE_m[data_PE_m.tipo_elem == 'NO_URB'].vmed.reset_index(drop=True) # vmed basada solo en puntos no urbanos
  data_PE_m.drop('tipo_elem', axis=1, inplace=True)
  data_PE_m = data_PE.groupby(['fecha'], as_index=False).agg({'intensidad':'mean', 'ocupacion':'mean', 'carga':'mean'})
  data_PE_m['vmed_noURB'] = vmed_noURB
  data_PE_m.insert(0,'estacion','PLAZA ELIPTICA')
  print(data_PE_m.head())
  print(data_PE_m.isnull().sum())

  # Mes
  data_month = pd.concat([data_B_m, data_CC_m, data_R_m, data_PE_m])
  data_month.to_csv('/content/drive/MyDrive/madrid_traffic_data/mod/2023/' + e + '_mod_points.csv', index=False) # guardar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3582506 entries, 0 to 3582505
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 246.0+ MB
None
id                        0
fecha                     0
tipo_elem                 0
intensidad                0
ocupacion              2964
carga                     0
vmed                    733
error                   690
periodo_integracion       0
dtype: int64
id            0
fecha         0
tipo_elem     0
intensidad    0
ocupacion     0
carga         0
vmed          0
dtype: int64
(3578809, 7)
BARAJAS
URB    5858
Name: tipo_elem, dtype

In [ ]:
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2017.csv') # chequear datos
print(d1.shape)
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2018.csv')
print(d2.shape)
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2019.csv')
print(d3.shape)
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2020.csv')
print(d4.shape)
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2021.csv')
print(d5.shape)
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2022.csv')
print(d6.shape)
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2023.csv')
print(d7.shape)

# por inspección visual fechas del 6, 7 y 8/12 - 2017 faltantes, acompletar en el dataset final.

(1460, 6)
(1460, 6)
(1460, 6)
(1464, 6)
(1460, 6)
(1460, 6)
(1460, 6)


***Unir años***

In [ ]:
d1 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2017.csv')
d2 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2018.csv')
d3 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2019.csv')
d4 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2020.csv')
d5 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2021.csv')
d6 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2022.csv')
d7 = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/years/m_traffic_points_2023.csv')

y = pd.concat([d1, d2, d3, d4, d5, d6, d7], axis=0, ignore_index=True)

y.to_csv('/content/drive/MyDrive/madrid_traffic_data/m_traffic_points_x_est_all.csv', index=False)
print(y.info())
y.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10224 entries, 0 to 10223
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    10224 non-null  object 
 1   fecha       10224 non-null  object 
 2   intensidad  10224 non-null  float64
 3   ocupacion   10224 non-null  float64
 4   carga       10224 non-null  float64
 5   vmed_noURB  9980 non-null   float64
dtypes: float64(4), object(2)
memory usage: 479.4+ KB
None


,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
10219,PLAZA ELIPTICA,2023-12-27,442.610851,7.392163,14.070328,44.468531
10220,PLAZA ELIPTICA,2023-12-28,452.437417,7.582276,14.243825,43.262400
10221,PLAZA ELIPTICA,2023-12-29,448.369909,7.503702,14.368563,44.952612
10222,PLAZA ELIPTICA,2023-12-30,368.677090,5.912876,12.088462,46.555256
10223,PLAZA ELIPTICA,2023-12-31,318.645963,5.065637,10.594595,45.380719


***Últimas modificaciones***

- Eliminar columna vmed_noURB para unificar datos entre estaciones.
- Añadir columna dia_tipo ('LAB' / 'No_LAB').
- Añadir columna mes.

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/m_traffic_points_x_est_all.csv')
data

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
0,BARAJAS,2017-01-01,156.769704,4.503079,15.968596,0.000000
1,BARAJAS,2017-01-02,226.734193,6.647023,23.390424,0.000000
2,BARAJAS,2017-01-03,238.387793,7.379162,24.723798,0.000000
3,BARAJAS,2017-01-04,238.496010,8.069368,25.219767,0.000000
4,BARAJAS,2017-01-05,234.923786,7.814997,24.885065,0.000000
...,...,...,...,...,...,...
10219,PLAZA ELIPTICA,2023-12-27,442.610851,7.392163,14.070328,44.468531
10220,PLAZA ELIPTICA,2023-12-28,452.437417,7.582276,14.243825,43.262400
10221,PLAZA ELIPTICA,2023-12-29,448.369909,7.503702,14.368563,44.952612
10222,PLAZA ELIPTICA,2023-12-30,368.677090,5.912876,12.088462,46.555256


In [ ]:
# usado más arriba para completar 12/17
fill = data[(data.estacion == 'BARAJAS') & (data.fecha == '2018-12-06')]
fill_b = fill.append(data[(data.estacion == 'BARAJAS') & (data.fecha == '2018-12-07')]).append(data[(data.estacion == 'BARAJAS') & (data.fecha == '2018-12-08')])
fill_b

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
2795,BARAJAS,2018-12-06,190.292249,5.348793,16.450445,0.0
2796,BARAJAS,2018-12-07,224.193982,6.758003,19.911652,0.0
2797,BARAJAS,2018-12-08,181.764217,5.291374,15.947604,0.0


In [ ]:
# usado más arriba para completar 12/17
fill = data[(data.estacion == 'CUATRO CAMINOS') & (data.fecha == '2018-12-06')]
fill_cc = fill.append(data[(data.estacion == 'CUATRO CAMINOS') & (data.fecha == '2018-12-07')]).append(data[(data.estacion == 'CUATRO CAMINOS') & (data.fecha == '2018-12-08')])
fill_cc

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
2826,CUATRO CAMINOS,2018-12-06,332.476688,3.727353,14.773655,0.0
2827,CUATRO CAMINOS,2018-12-07,427.993395,4.933046,19.150953,0.0
2828,CUATRO CAMINOS,2018-12-08,329.678911,3.699637,14.613476,0.0


In [ ]:
# usado más arriba para completar 12/17
fill_b.replace(['2018-12-06','2018-12-07','2018-12-08'],['2017-12-06','2017-12-07','2017-12-08'], inplace=True)
fill_cc.replace(['2018-12-06','2018-12-07','2018-12-08'],['2017-12-06','2017-12-07','2017-12-08'], inplace=True)

,estacion,fecha,intensidad,ocupacion,carga,vmed_noURB
2826,CUATRO CAMINOS,2017-12-06,332.476688,3.727353,14.773655,0.0
2827,CUATRO CAMINOS,2017-12-07,427.993395,4.933046,19.150953,0.0
2828,CUATRO CAMINOS,2017-12-08,329.678911,3.699637,14.613476,0.0


In [ ]:
data.drop(['vmed_noURB', ], axis=1, inplace=True)

In [ ]:
data.insert(2, 'mes', pd.to_datetime(data.fecha).dt.month)

In [ ]:
data.insert(3, 'dia_tipo', None)

In [ ]:
festivos = pd.read_excel('/content/drive/MyDrive/madrid_traffic_data/calendario.xlsx')
festivos.dropna(axis=0, inplace=True)
print(festivos)
dates = festivos.fecha.dt.date
dates.astype(str)

          fecha tipo_dia
5    2017-01-06  Festivo
78   2017-03-20  Festivo
102  2017-04-13  Festivo
103  2017-04-14  Festivo
120  2017-05-01  Festivo
...         ...      ...
2495 2023-11-01  Festivo
2503 2023-11-09  Festivo
2530 2023-12-06  Festivo
2532 2023-12-08  Festivo
2549 2023-12-25  Festivo

[98 rows x 2 columns]


5       2017-01-06
78      2017-03-20
102     2017-04-13
103     2017-04-14
120     2017-05-01
           ...    
2495    2023-11-01
2503    2023-11-09
2530    2023-12-06
2532    2023-12-08
2549    2023-12-25
Name: fecha, Length: 98, dtype: object

In [ ]:
index = data[data.fecha.isin(dates.astype(str))].index
for i in index:
  data.iloc[i,3] = 'No_LAB'

In [ ]:
data.head(30)

,estacion,fecha,mes,dia_tipo,intensidad,ocupacion,carga
0,BARAJAS,2017-01-01,1,None,156.769704,4.503079,15.968596
1,BARAJAS,2017-01-02,1,None,226.734193,6.647023,23.390424
2,BARAJAS,2017-01-03,1,None,238.387793,7.379162,24.723798
3,BARAJAS,2017-01-04,1,None,238.496010,8.069368,25.219767
4,BARAJAS,2017-01-05,1,None,234.923786,7.814997,24.885065
5,BARAJAS,2017-01-06,1,No_LAB,159.745074,5.233374,16.700739
6,BARAJAS,2017-01-07,1,None,180.033909,6.118989,19.157830
7,BARAJAS,2017-01-08,1,None,166.569839,5.429543,17.523486
8,BARAJAS,2017-01-09,1,None,272.492336,8.850399,28.452483
9,BARAJAS,2017-01-10,1,None,274.380542,8.655172,28.509852


In [ ]:
data.insert(4, 'dia_w', pd.to_datetime(data.fecha).dt.day_name())
data.dia_w.replace(['Saturday', 'Sunday'], 'No_LAB', inplace=True) # sábados y domingos
data.dia_w.replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], None, inplace=True)
data.dia_w.unique()

array(['No_LAB', None], dtype=object)

In [ ]:
data.head(30)

,estacion,fecha,mes,dia_tipo,dia_w,intensidad,ocupacion,carga
0,BARAJAS,2017-01-01,1,None,No_LAB,156.769704,4.503079,15.968596
1,BARAJAS,2017-01-02,1,None,None,226.734193,6.647023,23.390424
2,BARAJAS,2017-01-03,1,None,None,238.387793,7.379162,24.723798
3,BARAJAS,2017-01-04,1,None,None,238.496010,8.069368,25.219767
4,BARAJAS,2017-01-05,1,None,None,234.923786,7.814997,24.885065
5,BARAJAS,2017-01-06,1,No_LAB,None,159.745074,5.233374,16.700739
6,BARAJAS,2017-01-07,1,None,No_LAB,180.033909,6.118989,19.157830
7,BARAJAS,2017-01-08,1,None,No_LAB,166.569839,5.429543,17.523486
8,BARAJAS,2017-01-09,1,None,None,272.492336,8.850399,28.452483
9,BARAJAS,2017-01-10,1,None,None,274.380542,8.655172,28.509852


In [ ]:
data.dia_tipo.fillna(data.dia_w, inplace=True)
data.head(30)

,estacion,fecha,mes,dia_tipo,dia_w,intensidad,ocupacion,carga
0,BARAJAS,2017-01-01,1,No_LAB,No_LAB,156.769704,4.503079,15.968596
1,BARAJAS,2017-01-02,1,None,None,226.734193,6.647023,23.390424
2,BARAJAS,2017-01-03,1,None,None,238.387793,7.379162,24.723798
3,BARAJAS,2017-01-04,1,None,None,238.496010,8.069368,25.219767
4,BARAJAS,2017-01-05,1,None,None,234.923786,7.814997,24.885065
5,BARAJAS,2017-01-06,1,No_LAB,None,159.745074,5.233374,16.700739
6,BARAJAS,2017-01-07,1,No_LAB,No_LAB,180.033909,6.118989,19.157830
7,BARAJAS,2017-01-08,1,No_LAB,No_LAB,166.569839,5.429543,17.523486
8,BARAJAS,2017-01-09,1,None,None,272.492336,8.850399,28.452483
9,BARAJAS,2017-01-10,1,None,None,274.380542,8.655172,28.509852


In [ ]:
data.dia_tipo.fillna('LAB', inplace=True)
data.head(30)

,estacion,fecha,mes,dia_tipo,dia_w,intensidad,ocupacion,carga
0,BARAJAS,2017-01-01,1,No_LAB,No_LAB,156.769704,4.503079,15.968596
1,BARAJAS,2017-01-02,1,LAB,None,226.734193,6.647023,23.390424
2,BARAJAS,2017-01-03,1,LAB,None,238.387793,7.379162,24.723798
3,BARAJAS,2017-01-04,1,LAB,None,238.496010,8.069368,25.219767
4,BARAJAS,2017-01-05,1,LAB,None,234.923786,7.814997,24.885065
5,BARAJAS,2017-01-06,1,No_LAB,None,159.745074,5.233374,16.700739
6,BARAJAS,2017-01-07,1,No_LAB,No_LAB,180.033909,6.118989,19.157830
7,BARAJAS,2017-01-08,1,No_LAB,No_LAB,166.569839,5.429543,17.523486
8,BARAJAS,2017-01-09,1,LAB,None,272.492336,8.850399,28.452483
9,BARAJAS,2017-01-10,1,LAB,None,274.380542,8.655172,28.509852


In [ ]:
data.drop('dia_w', axis=1, inplace=True)
data.head(30)

,estacion,fecha,mes,dia_tipo,intensidad,ocupacion,carga
0,BARAJAS,2017-01-01,1,No_LAB,156.769704,4.503079,15.968596
1,BARAJAS,2017-01-02,1,LAB,226.734193,6.647023,23.390424
2,BARAJAS,2017-01-03,1,LAB,238.387793,7.379162,24.723798
3,BARAJAS,2017-01-04,1,LAB,238.496010,8.069368,25.219767
4,BARAJAS,2017-01-05,1,LAB,234.923786,7.814997,24.885065
5,BARAJAS,2017-01-06,1,No_LAB,159.745074,5.233374,16.700739
6,BARAJAS,2017-01-07,1,No_LAB,180.033909,6.118989,19.157830
7,BARAJAS,2017-01-08,1,No_LAB,166.569839,5.429543,17.523486
8,BARAJAS,2017-01-09,1,LAB,272.492336,8.850399,28.452483
9,BARAJAS,2017-01-10,1,LAB,274.380542,8.655172,28.509852


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data.shape

(10224, 7)

In [ ]:
data.to_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_all_final.csv', index=False)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10224 entries, 0 to 10223
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   estacion    10224 non-null  object 
 1   fecha       10224 non-null  object 
 2   mes         10224 non-null  int64  
 3   dia_tipo    10224 non-null  object 
 4   intensidad  10224 non-null  float64
 5   ocupacion   10224 non-null  float64
 6   carga       10224 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 559.2+ KB


In [ ]:
data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
estacion,10224,4,BARAJAS,2556,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fecha,10224,2556,2017-01-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mes,10224.0,NaN,NaN,NaN,6.524257,3.448507,1.0,4.0,7.0,10.0,12.0
dia_tipo,10224,2,LAB,6948,NaN,NaN,NaN,NaN,NaN,NaN,NaN
intensidad,10224.0,NaN,NaN,NaN,381.21913,141.410208,19.915416,271.24613,390.975628,492.882219,1186.258638
ocupacion,10224.0,NaN,NaN,NaN,6.128937,2.04864,0.342857,4.689269,6.100292,7.653679,15.572387
carga,10224.0,NaN,NaN,NaN,18.833822,5.412244,1.437926,15.178395,18.657494,22.902749,37.41723


In [103]:
data = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_all_final.csv')
data_b = data[data.estacion == 'BARAJAS']
data_b

data_b.to_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_Barajas_final.csv', index=False)

In [ ]:
data_b.shape


(2556, 7)

In [101]:
data = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_all_final.csv')
data_c = data[data.estacion == 'CUATRO CAMINOS']
data_c

data_c.to_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_CuatroCaminos_final.csv', index=False)

In [ ]:
data_c.shape

(2556, 7)

In [106]:
data = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_all_final.csv')
data_r = data[data.estacion == 'RETIRO']
data_r

data_r.to_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_Retiro_final.csv', index=False)

In [ ]:
data_r.shape

(2556, 7)

In [108]:
data = pd.read_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_all_final.csv')
data_p = data[data.estacion == 'PLAZA ELIPTICA']
data_p

data_p.to_csv('/content/drive/MyDrive/madrid_traffic_data/madrid_traffic_points_x_est_PlazaEliptica_final.csv', index=False)

In [ ]:
data_p.shape

(2556, 7)

In [109]:
data.dia_tipo.unique()

array(['No_LAB', 'LAB'], dtype=object)